In [1]:
import pennylane as qml
from pennylane import numpy as np

import torch

In [3]:
dev = qml.device("default.qubit", wires=4)

@qml.qnode(dev)
def circuit(n_wires):
    for i in range(n_wires):
        qml.Hadamard(i)
    return qml.probs(range(n_wires))

with qml.Tracker(dev) as tracker:
    for i in range(1, 5):
        circuit(i)

In [5]:
resources = tracker.history["resources"]
resources[0]

wires: 1
gates: 1
depth: 1
shots: Shots(total=None)
gate_types:
{'Hadamard': 1}
gate_sizes:
{1: 1}


In [6]:
[r.num_wires for r in resources]

[1, 2, 3, 4]

In [7]:
n_wires = 50
dev = qml.device("null.qubit", wires=n_wires)

weight_shape = qml.StronglyEntanglingLayers.shape(2, n_wires)
weights = np.random.random(weight_shape, requires_grad=True)

@qml.qnode(dev, diff_method="parameter-shift")
def circuit(weights):
    qml.StronglyEntanglingLayers(weights, wires=range(n_wires))
    return qml.expval(qml.PauliZ(0))

with qml.Tracker(dev) as tracker:
    qml.grad(circuit)(weights)

In [8]:
tracker.totals

{'executions': 451, 'batches': 2, 'batch_len': 451}

In [10]:
class CustomOp(qml.resource.ResourcesOperation):
    def resources(self):
        n = len(self.wires)
        r = qml.resource.Resources(
            num_wires=n,
            num_gates=n ** 2,
            depth=5,
        )
        return r

In [11]:
wires = [0, 1, 2]
c = CustomOp(wires)
c.resources()

wires: 3
gates: 9
depth: 5
shots: Shots(total=None)
gate_types:
{}
gate_sizes:
{}


In [12]:
dev = qml.device("default.qubit", wires=wires)

@qml.qnode(dev)
def circ():
    qml.PauliZ(wires=0)
    CustomOp(wires)
    return qml.state()

In [13]:
specs = qml.specs(circ)()
specs["resources"].depth

6

In [14]:
dev = qml.device('default.qubit', wires=2)

@qml.qnode(dev)
def circuit(param):
    qml.RY(param, wires=0)
    qml.CNOT(wires=[0, 1])
    return qml.state()

In [15]:
trace_distance_circuit = qml.qinfo.trace_distance(circuit, circuit, wires0=[0], wires1=[0])

In [16]:
x, y = np.array(0.4), np.array(0.6)

In [17]:
trace_distance_circuit((x,), (y,))

0.047862689546603415

In [18]:
wires = range(2)
dev = qml.device("default.qutrit", wires=wires)

@qml.qnode(dev)
def qutrit_circuit():
    qml.QutritBasisState([1, 1], wires=wires)
    qml.TAdd(wires=wires)
    return qml.probs(wires=1)

In [19]:
qutrit_circuit()

array([0., 0., 1.])

In [20]:
from pennylane.transforms import one_qubit_decomposition

In [22]:
n_qubits = 2
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def qnode(inputs, weights):
    qml.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.BasicEntanglerLayers(weights, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(wires=i)) for i in range(n_qubits)]

n_layers = 6
weight_shapes = {"weights": (n_layers, n_qubits)}
qlayer = qml.qnn.TorchLayer(qnode, weight_shapes)

In [24]:
batch_size = 10
inputs = torch.rand((batch_size, n_qubits))
qlayer(inputs)
dev.num_executions == 1

True

In [25]:
print(qml.draw(qlayer, show_matrices=False)(inputs))

0: ─╭AngleEmbedding(M0)─╭BasicEntanglerLayers(M1)─┤  <Z>
1: ─╰AngleEmbedding(M0)─╰BasicEntanglerLayers(M1)─┤  <Z>


In [26]:
dev = qml.device("default.qubit", wires=2)
@qml.qnode(dev)
def circuit(state):
    return qml.expval(qml.Projector(state, wires=[0, 1]))
zero_state = [0, 0]
plusplus_state = np.array([1, 1, 1, 1]) / 2

In [27]:
circuit(zero_state)

tensor(1., requires_grad=True)

In [28]:
circuit(plusplus_state)

tensor(0.25, requires_grad=True)